In [1]:
import cv2
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import csv
import time
import pickle

# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
#CSV file reading
"""
Generating dictionaries
for photo_id ---> business_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))

"""
Creates no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
def id_to_label(photo_ids):
    photo_labels = np.empty(shape=[0,9])
    for id in photo_ids:
        lst_labels = bus_to_labels_dict[photo_to_bus_dict[id]]
        ls = lst_labels[0].split(' ')
        labels = [0]*9
        for label in ls:
            if label !='':
                labels[int(label)] = 1
        labels = np.array(labels).reshape(1,-1)
        photo_labels = np.append(photo_labels,labels,axis=0)
    return photo_labels

GPU is active
Column names are photo_id, business_id
Processed 234843 lines.
234842
Column names are business_id, labels
Processed 2001 lines.
2000


In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier


inputpath = 'alexNet_fc7/'  #Input folder to read the alexNet features extracted

random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, probability=True, random_state=random_state))
    
i = 1
file_list = sorted(os.listdir(inputpath))
train_list = file_list
np_data = np.empty(shape=[0, 4096])
np_label = np.empty(shape=[0,9])

for file_name in train_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['features']
    print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    np_data = np.append(np_data, data, axis = 0)
    np_label = np.append(np_label, label, axis = 0)
    i = i + 1
    
classifier.fit(np_data, np_label)    

10.jpg.mat - 1
(1, 4096)
10002.jpg.mat - 2
(1, 4096)
10004.jpg.mat - 3
(1, 4096)
10008.jpg.mat - 4
(1, 4096)
10009.jpg.mat - 5
(1, 4096)
1001.jpg.mat - 6
(1, 4096)
10011.jpg.mat - 7
(1, 4096)
10014.jpg.mat - 8
(1, 4096)
10016.jpg.mat - 9
(1, 4096)
1002.jpg.mat - 10
(1, 4096)
10020.jpg.mat - 11
(1, 4096)
10022.jpg.mat - 12
(1, 4096)
10023.jpg.mat - 13
(1, 4096)
10024.jpg.mat - 14
(1, 4096)
1003.jpg.mat - 15
(1, 4096)
10031.jpg.mat - 16
(1, 4096)
10032.jpg.mat - 17
(1, 4096)
10035.jpg.mat - 18
(1, 4096)
10036.jpg.mat - 19
(1, 4096)
10037.jpg.mat - 20
(1, 4096)
10041.jpg.mat - 21
(1, 4096)
10045.jpg.mat - 22
(1, 4096)
10046.jpg.mat - 23
(1, 4096)
10050.jpg.mat - 24
(1, 4096)
10052.jpg.mat - 25
(1, 4096)
10053.jpg.mat - 26
(1, 4096)
10056.jpg.mat - 27
(1, 4096)
10057.jpg.mat - 28
(1, 4096)
10058.jpg.mat - 29
(1, 4096)
10061.jpg.mat - 30
(1, 4096)
10062.jpg.mat - 31
(1, 4096)
10063.jpg.mat - 32
(1, 4096)
10065.jpg.mat - 33
(1, 4096)
10069.jpg.mat - 34
(1, 4096)
10070.jpg.mat - 35
(1, 4096)


10528.jpg.mat - 280
(1, 4096)
10529.jpg.mat - 281
(1, 4096)
10532.jpg.mat - 282
(1, 4096)
10534.jpg.mat - 283
(1, 4096)
10535.jpg.mat - 284
(1, 4096)
10536.jpg.mat - 285
(1, 4096)
10538.jpg.mat - 286
(1, 4096)
1054.jpg.mat - 287
(1, 4096)
10543.jpg.mat - 288
(1, 4096)
10544.jpg.mat - 289
(1, 4096)
10546.jpg.mat - 290
(1, 4096)
10547.jpg.mat - 291
(1, 4096)
10548.jpg.mat - 292
(1, 4096)
1055.jpg.mat - 293
(1, 4096)
10550.jpg.mat - 294
(1, 4096)
10554.jpg.mat - 295
(1, 4096)
10556.jpg.mat - 296
(1, 4096)
10557.jpg.mat - 297
(1, 4096)
10558.jpg.mat - 298
(1, 4096)
1056.jpg.mat - 299
(1, 4096)
10560.jpg.mat - 300
(1, 4096)
10562.jpg.mat - 301
(1, 4096)
10564.jpg.mat - 302
(1, 4096)
10566.jpg.mat - 303
(1, 4096)
10567.jpg.mat - 304
(1, 4096)
10569.jpg.mat - 305
(1, 4096)
10571.jpg.mat - 306
(1, 4096)
10572.jpg.mat - 307
(1, 4096)
10573.jpg.mat - 308
(1, 4096)
10576.jpg.mat - 309
(1, 4096)
10577.jpg.mat - 310
(1, 4096)
10579.jpg.mat - 311
(1, 4096)
1058.jpg.mat - 312
(1, 4096)
10581.jpg.mat 

10991.jpg.mat - 557
(1, 4096)
10993.jpg.mat - 558
(1, 4096)
10994.jpg.mat - 559
(1, 4096)
10995.jpg.mat - 560
(1, 4096)
10996.jpg.mat - 561
(1, 4096)
10997.jpg.mat - 562
(1, 4096)
10999.jpg.mat - 563
(1, 4096)
1100.jpg.mat - 564
(1, 4096)
11000.jpg.mat - 565
(1, 4096)
11005.jpg.mat - 566
(1, 4096)
11006.jpg.mat - 567
(1, 4096)
11007.jpg.mat - 568
(1, 4096)
11011.jpg.mat - 569
(1, 4096)
11012.jpg.mat - 570
(1, 4096)
11013.jpg.mat - 571
(1, 4096)
11014.jpg.mat - 572
(1, 4096)
11017.jpg.mat - 573
(1, 4096)
11018.jpg.mat - 574
(1, 4096)
1102.jpg.mat - 575
(1, 4096)
11020.jpg.mat - 576
(1, 4096)
11023.jpg.mat - 577
(1, 4096)
11024.jpg.mat - 578
(1, 4096)
11027.jpg.mat - 579
(1, 4096)
11028.jpg.mat - 580
(1, 4096)
1103.jpg.mat - 581
(1, 4096)
11030.jpg.mat - 582
(1, 4096)
11033.jpg.mat - 583
(1, 4096)
11036.jpg.mat - 584
(1, 4096)
11039.jpg.mat - 585
(1, 4096)
1104.jpg.mat - 586
(1, 4096)
11040.jpg.mat - 587
(1, 4096)
11041.jpg.mat - 588
(1, 4096)
11043.jpg.mat - 589
(1, 4096)
11046.jpg.mat 

11472.jpg.mat - 833
(1, 4096)
11473.jpg.mat - 834
(1, 4096)
11474.jpg.mat - 835
(1, 4096)
11476.jpg.mat - 836
(1, 4096)
11477.jpg.mat - 837
(1, 4096)
11480.jpg.mat - 838
(1, 4096)
11481.jpg.mat - 839
(1, 4096)
11482.jpg.mat - 840
(1, 4096)
11483.jpg.mat - 841
(1, 4096)
11484.jpg.mat - 842
(1, 4096)
11486.jpg.mat - 843
(1, 4096)
11489.jpg.mat - 844
(1, 4096)
11491.jpg.mat - 845
(1, 4096)
11495.jpg.mat - 846
(1, 4096)
11497.jpg.mat - 847
(1, 4096)
11498.jpg.mat - 848
(1, 4096)
11499.jpg.mat - 849
(1, 4096)
1150.jpg.mat - 850
(1, 4096)
11501.jpg.mat - 851
(1, 4096)
11502.jpg.mat - 852
(1, 4096)
11503.jpg.mat - 853
(1, 4096)
11504.jpg.mat - 854
(1, 4096)
11505.jpg.mat - 855
(1, 4096)
11506.jpg.mat - 856
(1, 4096)
11508.jpg.mat - 857
(1, 4096)
1151.jpg.mat - 858
(1, 4096)
11510.jpg.mat - 859
(1, 4096)
11514.jpg.mat - 860
(1, 4096)
11518.jpg.mat - 861
(1, 4096)
11521.jpg.mat - 862
(1, 4096)
11522.jpg.mat - 863
(1, 4096)
11523.jpg.mat - 864
(1, 4096)
11525.jpg.mat - 865
(1, 4096)
11526.jpg.ma

11968.jpg.mat - 1110
(1, 4096)
1197.jpg.mat - 1111
(1, 4096)
11971.jpg.mat - 1112
(1, 4096)
11972.jpg.mat - 1113
(1, 4096)
11977.jpg.mat - 1114
(1, 4096)
1198.jpg.mat - 1115
(1, 4096)
11982.jpg.mat - 1116
(1, 4096)
11984.jpg.mat - 1117
(1, 4096)
11986.jpg.mat - 1118
(1, 4096)
11987.jpg.mat - 1119
(1, 4096)
1199.jpg.mat - 1120
(1, 4096)
11990.jpg.mat - 1121
(1, 4096)
11993.jpg.mat - 1122
(1, 4096)
11996.jpg.mat - 1123
(1, 4096)
12.jpg.mat - 1124
(1, 4096)
1200.jpg.mat - 1125
(1, 4096)
12000.jpg.mat - 1126
(1, 4096)
12001.jpg.mat - 1127
(1, 4096)
12002.jpg.mat - 1128
(1, 4096)
12005.jpg.mat - 1129
(1, 4096)
12006.jpg.mat - 1130
(1, 4096)
12007.jpg.mat - 1131
(1, 4096)
12008.jpg.mat - 1132
(1, 4096)
12009.jpg.mat - 1133
(1, 4096)
12010.jpg.mat - 1134
(1, 4096)
12013.jpg.mat - 1135
(1, 4096)
12016.jpg.mat - 1136
(1, 4096)
12017.jpg.mat - 1137
(1, 4096)
12019.jpg.mat - 1138
(1, 4096)
1202.jpg.mat - 1139
(1, 4096)
12022.jpg.mat - 1140
(1, 4096)
12026.jpg.mat - 1141
(1, 4096)
12027.jpg.mat - 

...

79628.jpg.mat - 38712
(1, 4096)
79629.jpg.mat - 38713
(1, 4096)
7963.jpg.mat - 38714
(1, 4096)
79630.jpg.mat - 38715
(1, 4096)
79632.jpg.mat - 38716
(1, 4096)
79633.jpg.mat - 38717
(1, 4096)
79635.jpg.mat - 38718
(1, 4096)
79636.jpg.mat - 38719
(1, 4096)
79638.jpg.mat - 38720
(1, 4096)
79639.jpg.mat - 38721
(1, 4096)
7964.jpg.mat - 38722
(1, 4096)
79641.jpg.mat - 38723
(1, 4096)
79644.jpg.mat - 38724
(1, 4096)
79645.jpg.mat - 38725
(1, 4096)
79646.jpg.mat - 38726
(1, 4096)
79648.jpg.mat - 38727
(1, 4096)
79649.jpg.mat - 38728
(1, 4096)
79651.jpg.mat - 38729
(1, 4096)
79653.jpg.mat - 38730
(1, 4096)
79654.jpg.mat - 38731
(1, 4096)
79656.jpg.mat - 38732
(1, 4096)
7966.jpg.mat - 38733
(1, 4096)
79660.jpg.mat - 38734
(1, 4096)
79662.jpg.mat - 38735
(1, 4096)
79665.jpg.mat - 38736
(1, 4096)
79666.jpg.mat - 38737
(1, 4096)
79669.jpg.mat - 38738
(1, 4096)
7967.jpg.mat - 38739
(1, 4096)
79672.jpg.mat - 38740
(1, 4096)
79673.jpg.mat - 38741
(1, 4096)
79675.jpg.mat - 38742
(1, 4096)
79676.jpg.ma

8141.jpg.mat - 38972
(1, 4096)
8145.jpg.mat - 38973
(1, 4096)
8146.jpg.mat - 38974
(1, 4096)
8147.jpg.mat - 38975
(1, 4096)
8148.jpg.mat - 38976
(1, 4096)
8149.jpg.mat - 38977
(1, 4096)
8150.jpg.mat - 38978
(1, 4096)
8153.jpg.mat - 38979
(1, 4096)
8154.jpg.mat - 38980
(1, 4096)
8158.jpg.mat - 38981
(1, 4096)
8159.jpg.mat - 38982
(1, 4096)
816.jpg.mat - 38983
(1, 4096)
8161.jpg.mat - 38984
(1, 4096)
8162.jpg.mat - 38985
(1, 4096)
8165.jpg.mat - 38986
(1, 4096)
8166.jpg.mat - 38987
(1, 4096)
8167.jpg.mat - 38988
(1, 4096)
8168.jpg.mat - 38989
(1, 4096)
8169.jpg.mat - 38990
(1, 4096)
817.jpg.mat - 38991
(1, 4096)
8170.jpg.mat - 38992
(1, 4096)
8173.jpg.mat - 38993
(1, 4096)
8179.jpg.mat - 38994
(1, 4096)
8180.jpg.mat - 38995
(1, 4096)
8183.jpg.mat - 38996
(1, 4096)
8184.jpg.mat - 38997
(1, 4096)
8186.jpg.mat - 38998
(1, 4096)
8188.jpg.mat - 38999
(1, 4096)
8189.jpg.mat - 39000
(1, 4096)
8191.jpg.mat - 39001
(1, 4096)
8193.jpg.mat - 39002
(1, 4096)
8196.jpg.mat - 39003
(1, 4096)
8199.jpg.m

8607.jpg.mat - 39238
(1, 4096)
861.jpg.mat - 39239
(1, 4096)
8610.jpg.mat - 39240
(1, 4096)
8612.jpg.mat - 39241
(1, 4096)
8613.jpg.mat - 39242
(1, 4096)
8615.jpg.mat - 39243
(1, 4096)
8619.jpg.mat - 39244
(1, 4096)
862.jpg.mat - 39245
(1, 4096)
8621.jpg.mat - 39246
(1, 4096)
8622.jpg.mat - 39247
(1, 4096)
8623.jpg.mat - 39248
(1, 4096)
8624.jpg.mat - 39249
(1, 4096)
8625.jpg.mat - 39250
(1, 4096)
8626.jpg.mat - 39251
(1, 4096)
8627.jpg.mat - 39252
(1, 4096)
8630.jpg.mat - 39253
(1, 4096)
8631.jpg.mat - 39254
(1, 4096)
8635.jpg.mat - 39255
(1, 4096)
8637.jpg.mat - 39256
(1, 4096)
8639.jpg.mat - 39257
(1, 4096)
8643.jpg.mat - 39258
(1, 4096)
8644.jpg.mat - 39259
(1, 4096)
8645.jpg.mat - 39260
(1, 4096)
8646.jpg.mat - 39261
(1, 4096)
865.jpg.mat - 39262
(1, 4096)
8652.jpg.mat - 39263
(1, 4096)
8653.jpg.mat - 39264
(1, 4096)
8654.jpg.mat - 39265
(1, 4096)
8657.jpg.mat - 39266
(1, 4096)
8658.jpg.mat - 39267
(1, 4096)
866.jpg.mat - 39268
(1, 4096)
8662.jpg.mat - 39269
(1, 4096)
8664.jpg.mat

9082.jpg.mat - 39504
(1, 4096)
9085.jpg.mat - 39505
(1, 4096)
9086.jpg.mat - 39506
(1, 4096)
9087.jpg.mat - 39507
(1, 4096)
9088.jpg.mat - 39508
(1, 4096)
9089.jpg.mat - 39509
(1, 4096)
9090.jpg.mat - 39510
(1, 4096)
9092.jpg.mat - 39511
(1, 4096)
9093.jpg.mat - 39512
(1, 4096)
9095.jpg.mat - 39513
(1, 4096)
9096.jpg.mat - 39514
(1, 4096)
9097.jpg.mat - 39515
(1, 4096)
9098.jpg.mat - 39516
(1, 4096)
91.jpg.mat - 39517
(1, 4096)
9100.jpg.mat - 39518
(1, 4096)
9102.jpg.mat - 39519
(1, 4096)
9105.jpg.mat - 39520
(1, 4096)
9107.jpg.mat - 39521
(1, 4096)
9109.jpg.mat - 39522
(1, 4096)
9110.jpg.mat - 39523
(1, 4096)
9113.jpg.mat - 39524
(1, 4096)
9114.jpg.mat - 39525
(1, 4096)
9115.jpg.mat - 39526
(1, 4096)
9117.jpg.mat - 39527
(1, 4096)
9118.jpg.mat - 39528
(1, 4096)
912.jpg.mat - 39529
(1, 4096)
9122.jpg.mat - 39530
(1, 4096)
9123.jpg.mat - 39531
(1, 4096)
9124.jpg.mat - 39532
(1, 4096)
9130.jpg.mat - 39533
(1, 4096)
9135.jpg.mat - 39534
(1, 4096)
9136.jpg.mat - 39535
(1, 4096)
9138.jpg.ma

9572.jpg.mat - 39770
(1, 4096)
9573.jpg.mat - 39771
(1, 4096)
9574.jpg.mat - 39772
(1, 4096)
9575.jpg.mat - 39773
(1, 4096)
9579.jpg.mat - 39774
(1, 4096)
958.jpg.mat - 39775
(1, 4096)
9580.jpg.mat - 39776
(1, 4096)
9581.jpg.mat - 39777
(1, 4096)
9583.jpg.mat - 39778
(1, 4096)
9585.jpg.mat - 39779
(1, 4096)
9586.jpg.mat - 39780
(1, 4096)
9587.jpg.mat - 39781
(1, 4096)
9588.jpg.mat - 39782
(1, 4096)
9589.jpg.mat - 39783
(1, 4096)
959.jpg.mat - 39784
(1, 4096)
9591.jpg.mat - 39785
(1, 4096)
9595.jpg.mat - 39786
(1, 4096)
9597.jpg.mat - 39787
(1, 4096)
9598.jpg.mat - 39788
(1, 4096)
960.jpg.mat - 39789
(1, 4096)
9600.jpg.mat - 39790
(1, 4096)
9601.jpg.mat - 39791
(1, 4096)
9603.jpg.mat - 39792
(1, 4096)
9604.jpg.mat - 39793
(1, 4096)
9607.jpg.mat - 39794
(1, 4096)
9609.jpg.mat - 39795
(1, 4096)
9611.jpg.mat - 39796
(1, 4096)
9612.jpg.mat - 39797
(1, 4096)
9615.jpg.mat - 39798
(1, 4096)
9616.jpg.mat - 39799
(1, 4096)
9618.jpg.mat - 39800
(1, 4096)
9620.jpg.mat - 39801
(1, 4096)
9628.jpg.ma

In [ ]:
### testing the model
inputpath = 'alexNet_fc7_test/'  #Input folder to read the alexNet features extracted

file_list = sorted(os.listdir(inputpath))
test_list = file_list
pred_list = []
label_list = []
i = 1
for file_name in test_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['features']
    print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    
    pred = classifier.predict(data)
    i = i + 1
    label_list.append(label)
    pred_list.append(pred)

In [ ]:
def verify_correctness(pred1, test_labels, threshold=0):
    cnt=0
    i = 1
    for x, y in zip(pred1, test_labels):
        x1= np.array(x)
        y1 = np.array(y)
        c=0
        for a, b in zip(x1, y1):
            if(a!=b): c+=1
        if c<=threshold:
            cnt+=1
        i = i + 1
    accuracy = cnt/len(test_labels)
    return accuracy

for p in pred_list[:50]:
    print(p)

In [3]:
print("Accuracy score for strict match is ", verify_correctness(pred_list, label_list, 0)*100)
print("Accuracy score for maximum of 1 mistmatch is ", verify_correctness(pred_list, label_list, 1)*100)
print("Accuracy score for maximum of 2 mistmatch is ", verify_correctness(pred_list, label_list, 2)*100)

Accuracy score for strict match is 43.2978
Accuracy score for maximum of 1 mistmatch is 49.1754
Accuracy score for maximum of 2 mistmatch is 57.2940
